# LACE Hospital Readmission Index


This calculation of the  LACE Hospital Readmission Index has been designed to work on the OMOP CDM v5.3 and developed in the University of North Carolina at Chapel Hill de-identified OMOP Research Data Repository, ORDR(D). Please see the README in the repo with important notes, clarifications, and assumptions.

Author: Nathan Foster

Copyright 2025, The University of North Carolina at Chapel Hill.  Permission is granted to use in accordance with the MIT license.  The code is licensed under the open-source MIT license. 

# Load packages, connect to the database, define functions

In [1]:
import sqlalchemy
import psycopg
import pandas as pd
import numpy as np
import datetime

from getpass import getpass
from itertools import groupby

In [2]:
# Prompt for username and password
user_name = input('enter username:')
user_password = getpass('enter password:')


# Configure connection
conn = psycopg.connect(
    host='od2-primary',
    dbname='ordrd',
    user=user_name,
    password=user_password)

enter username: jtfuchs
enter password: ········


In [3]:
cursor = conn.cursor()

In [4]:
def query_to_dataframe(query_string, column_names, d_types):
    '''
    Runs a SQL query on the database and returns the results
    as a pandas DataFrame.
    
    PARAMETERS
    query_string : string
        string formatted as a SQL command, to execute on the database
        
    column_name : list
        list of strings for desired names of returned columns
        length must match the number of columsn returned in the
        query        
        
        If you are selecting all (*) columns from a single table,
        this value can be None. In that case, this function will
        query the table to get the column names and use those to
        create the dataframe
        
    d_types : dictionary
        dictionary mapping the column names as keys to the data type
        as values for the resulting DataFrame
    
    RETURNS
        conditional. If error with query, returns
        
        query_result : None
        
        if query is successful, returns
        
        query_df : pandas dataframe
            DataFrame with column names set from column_names
    '''
    try:
        cursor.execute(query_string)
        query_result = cursor.fetchall()
    except:
        print("Error with SQL command")
        conn.rollback()
        query_result = None    

    if query_result is not None:
        if column_names is None:
            # First, get the table name
            table_name = extract_substring(query_string).strip()
            # Build the query to get the column names
            column_name_query = "select column_name from information_schema.columns where table_name = '{}';".format(table_name)
            # Execute the query
            cursor.execute(column_name_query)
            column_names_tuple = cursor.fetchall()
            # Convert tuples to strings
            column_names = [' '.join(item) for item in column_names_tuple]
        query_df = pd.DataFrame(query_result,columns=column_names).astype(d_types)
        return query_df
    
    return query_result

In [5]:
def extract_substring(query_string):
    '''
    Takes an input query string and returns the name of 
    the table that is being queried. Only works for a single
    table. 
    
    Example: if you are querying omop.measurement, this 
    will return measurement
    
    PARAMETERS
    query_string : string
        query string
        
    RETURNS
    table_string : string
        table that is being queried. Found after omop.
    '''
    # Find the index of "omop"
    start_index = query_string.find("omop")
    if start_index != -1:
        # Find the index of the next space after "omop"
        end_index = query_string.find(" ", start_index)
        if end_index != -1:
            # Extract the desired substring
            table_string = query_string[start_index + 4:end_index]
            # The substring will begin with a .
            # ignore that
            return table_string[1:]
    return None  # Return None if "omop" or space is not found

# Query Database 

Query the database to get all ER and inpatient visits in a DataFrame. 

- visit_concept_id 262 corresponds to an ER visit with subsequent inpatient stay. 
- visit_concept_id 9201 corresponds to an inpatient visit alone. 
- visit_concept_id 9203 corresponds to an ER visit alone. 

To limit the date range, add a line at the end of the visit_query:
and visit_start_date > '2022/01/01'

In [6]:
visit_query = ("""\
                    SELECT visit_occurrence_id, person_id, visit_concept_id, 
                        visit_start_date, visit_end_date
                    FROM omop.visit_occurrence
                    WHERE visit_concept_id IN (262, 9201, 9203); 
                    """)

In [7]:
column_names = ['visit_occurrence_id','person_id','visit_concept_id', 'visit_start_date', 'visit_end_date']

d_types = {
    'visit_occurrence_id': 'object',
    'person_id': 'object',
    'visit_concept_id': 'int64',
    'visit_start_date': 'datetime64[ns]',
    'visit_end_date': 'datetime64[ns]'
}

visit_df = query_to_dataframe(visit_query,column_names,d_types)

In [8]:
# Only inpatient visits (ER + inpatient or inpatient alone) 
inpatient_visits = visit_df[visit_df.visit_concept_id.isin([262, 9201])]

# Only ER visits (ER + inpatient or ER alone) 
er_visits = visit_df[visit_df.visit_concept_id.isin([262, 9203])]

# Calculate L: Length of Stay

A score is assigned based on the length of the inpatient stay.

In [9]:
def l_score_mapping(x):
    '''
    Converts number of days into the length of stay component of LACE
    
    PARAMETERS
    x : int
        length of stay
        
    RETURNS
    l_score : int
        score for the L component of LACE
    '''
    if x < 1:
        l_score = 0
    elif x == 1:
        l_score = 1
    elif x == 2:
        l_score = 2
    elif x == 3:
        l_score = 3
    elif ((x >= 4) & (x <= 6)):
        l_score = 4
    elif ((x >= 7) & (x <= 13)):
        l_score = 5
    else:
        l_score = 7
    return l_score

In [10]:
def get_l_score(df):
    '''
    Calculates a score based on the length of the inpatient stay
    
    PARAMETERS
    df : DataFrame
        dataframe with visit_end_date and visit_start_date as named columns
           
    RETURNS
    DataFrame
        dataframe with visit_occurrence_id, length of stay, and the score for the 
        L component of LACE
    '''
    # Calculate the length of stay
    df.loc[:,'length'] = (df['visit_end_date'] - df['visit_start_date'])
    df.loc[:,'length'] = df['length'].dt.days
    
    # Calculate the score for that length
    df.loc[:,'l_score'] = df['length'].apply(l_score_mapping)
    
    return df[["visit_occurrence_id", "length", "l_score"]]

In [11]:
length_of_stay = get_l_score(inpatient_visits)

/tmp/ipykernel_146/2185766201.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'length'] = (df['visit_end_date'] - df['visit_start_date'])
/tmp/ipykernel_146/2185766201.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'l_score'] = df['length'].apply(l_score_mapping)


# Calculate A: Acuity of Admission

Acuity of admission is inferred from the type of visit. An inpatient visit alone, *visit_concept_id* = 9201, is considered a non-emergent/non-acute admission. An ER visit followed by an inpatient visit, *visit_concept_id* = 262, is considered an emergent/acute admission. For a more detained disucssion, consult the README. 


In [12]:
def a_score_mapping(x):
    '''
    Using the visit_concept_id, returns an acuity of admission score
    3 = acute/emergent, 0 = non-acute/non-emergent
    
    PARAMETERS
    x : int
        integeger representing the OMOP Concept ID for the corresponding
        visit
    
    RETURNS
    a_score : int
        score for the A component of LACE
    '''
    if x == 262:
        a_score = 3
    else:
        a_score = 0
    return a_score

In [13]:
def get_a_score(df):
    '''
    Calculates a score based on the acuity of patient admission. 
    
    PARAMETERS
    df : DataFrame
        dataframe with visit_concept_id as a column
           
    RETURNS
    DataFrame
        dataframe with visit_occurrence_id and the score for the 
        A component of LACE
    '''
    df.loc[:,'a_score'] = df['visit_concept_id'].apply(a_score_mapping)
    
    return df[["visit_occurrence_id","a_score"]]

In [14]:
acuity_of_admission = get_a_score(inpatient_visits)

/tmp/ipykernel_146/1947189066.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'a_score'] = df['visit_concept_id'].apply(a_score_mapping)


# Calculate C: Charlson Comorbidity Index

Calculates the CCI for each inpatient visit to convert to the C component of LACE. See the README for details about this calculation.

### Helper functions for Charlson Comorbidity Index calculation

In [15]:
def split_list_to_dict(lst):
    """
    Splits a Python list into a dictionary where the keys are the length of each entry
    and the values are the corresponding entries.
    
    PARAMETERS
    lst : list
        list of strings
        
    RETURNS
    result_dict : dictionary
        dictionary where keys are the length of entries in lst, and values are entries
        in lst that correspond to that length
    """
    result_dict = {}
    for item in lst:
        key = len(item)
        if key not in result_dict:
            result_dict[key] = []
        result_dict[key].append(item)
    return result_dict

In [16]:
def add_new_row(df):
    """
    Adds a new row to the dataframe if the condition_source_value is 4373. This is to account
    for 437.3 and 437 being listed as two sepate indicators in Quan et al. (2005). 
    
    For patients with this condition, this function appends a row so that both formats
    are included. 
    
    PARAMETERS
    df : pandas DataFrame
    
    RETURNS
    new_df : pandas DataFrame
    new rows are appended at the bottom of the input dataframe
    """
    # Identify rows where condition_source_value is '4373'
    mask = df['condition_source_value'] == '4373'
    
    # Create a new DataFrame with the same person_id and condition_source_value = '437'
    new_row = df[mask].copy()
    new_row['condition_source_value'] = '437'
    
    # Append the new row to the original DataFrame
    new_df = pd.concat([df,new_row],ignore_index=True)
    
    return new_df

In [17]:
def calculate_score(condition_list,weight_col):
    """
    Calculates the CCI given a list of conditions a patient has. If a condition appears twice in the condition_df,
    then it will be counted twice here. 
      
    PARAMETERS
    condition_list : list
        list of conditions a patient has diagnoses for
        
    weight_col : string
        column in condition_df containing the weights for each condition group
        
    RETURNS
    total_score : int
        CCI for these conditions
    """
    total_score = 0
    
    # Iterate through the condition_df to go through,
    # condition by condition, and see if any values in the
    # condition list match any of the values for that particular
    # condition. If so, sum the score for that condition to 
    # the overall score
    for index, row in condition_df.iterrows():
        if bool(set(condition_list).intersection(set(row['icd']))):
            total_score += row[weight_col]
    return total_score

### Define Conditions

In [18]:
columns = ['icd','charlson87_weight']

condition_dictionary = {'myocardial infarction' : [['410','412','I21','I22','I252'],1],
                        'congestive heart failure' : [['39891','40201','40211','40291','40401','40403','40411','40413','40491','40493',
                                                       '4254','4255',
                                                        '4256','4257','4258','4259','428','I099','I110','I130','I132','I255','I420',
                                                       'I425','I426','I427',
                                                              'I428','I429','P290','I43','I50'],1],
                        'peripheral vascular disease' : [['0930','4373','4431','4432','4433','4434','4435','4436','4437','4438','4439',
                                                          '4471','5571','5579',
                                                                 'V434','440','441','I731','I738','I739','I771','I790','I792','K551',
                                                          'K558','K559','Z958','Z959',
                                                                 'I70','I71'],1],
                        'cerebrovascular disease' : [['430','431','432','433','434','435','436','437','438','36234','G45','G46','I60',
                                                      'I61','I62','I63',
                                                             'I64','I65','I66','I67','I68','I69','H340'],1],
                        'dementia' : [['29410','29411','3312','290','F051','G311','F00','F01','F02','F03','G30'],1],
                        'chronic pulmonary disease' : [['490','491','492','493','494','495','496','500','501','502','503','504','505',
                                                        '4168','4169','5064',
                                                               '5081','5088','J40','J41','J42','J43','J44','J45','J46','J47','J60','J61',
                                                        'J62','J63','J64','J65',
                                                               'J66','J67','I278','I279','J684','J701','J703'],1],
                        'connective tissue disease' : [['4465','7100','7101','7102','7103','7104','7140','7141','7142','7148','725','M315',
                                                        'M351','M353',
                                                                'M360','M05','M32','M33','M34','M06'],1],
                        'ulcer disease' : [['531','532','533','534','K25','K26','K27','K28'],1],
                        'mild liver disease' : [['07022','07023','07032','07033','07044','07054','0706','0709','5733','5734','5738','5739',
                                                 'V427','570',
                                                        '571','K700','K701','K702','K703','K709','K717','K713','K714','K715','K760','K762',
                                                 'K763','K764','K768',
                                                        'K769','Z944','B18','K73','K74'],1],
                        'diabetes without complications' : [['2500','2501','2502','2503','2508','2509','E100','E101','E106','E108','E109',
                                                             'E110','E111',
                                                                     'E116','E118','E119','E120','E121','E126','E128','E129','E130','E131',
                                                             'E136','E138','E139',
                                                                     'E140','E141','E146','E148','E149'],1],
                        'hemiplegia' : [['3341','3440','3441','3442','3443','3444','3445','3446','3449','342','343','G041','G114','G801',
                                         'G802','G830',
                                                'G831','G832','G833','G834','G839','G81','G82'],2],
                        'renal disease' : [['40301','40311','40391','40402','40403','40412','40413','40492','40493','5830','5831','5832',
                                            '5834','5836','5837',
                                              '5880','V420','V451','582','585','586','V56','N18','N19','N052','N053','N054','N055','N056',
                                            'N057','N250','I120',
                                              'I131','N032','N033','N034','N035','N036','N037','Z490','Z491','Z492','Z940','Z992'],2],
                        'diabetes with complications' : [['2504','2505','2506','2507','E102','E103','E104','E105','E107','E112','E113','E114',
                                                          'E115','E117',
                                                            'E122','E123','E124','E125','E127','E132','E133','E134','E135','E137','E142',
                                                          'E143','E144','E145',
                                                            'E147'],2],
                        'cancer' : [['140','141','142','143','144','145','146','147','148','149','150','151','152','153','154','155','156',
                                     '157','158','159',
                                       '160','161','162','163','164','165','170','171','172','174','175','176','179','180','181','182','183',
                                     '184','185','186',
                                       '187','188','189','190','191','192','193','194','195','200','201','202','203','204','205','206','207',
                                     '208','2386','C00',
                                       'C01','C02','C03','C04','C05','C06','C07','C08','C09','C10','C11','C12','C13','C14','C15','C16','C17',
                                     'C18','C19','C20',
                                       'C21','C22','C23','C24','C25','C26','C30','C31','C32','C33','C34','C37','C38','C39','C40','C41','C43',
                                     'C45','C46','C47',
                                       'C48','C49','C50','C51','C52','C53','C54','C55','C56','C57','C58','C60','C61','C62','C63','C64','C65',
                                     'C66','C67','C68',
                                       'C69','C70','C71','C72','C73','C74','C75','C76','C81','C82','C83','C84','C85','C88','C90','C91','C92',
                                     'C93','C94','C95',
                                       'C96','C97'],2],
                        'moderate or severe liver disease' : [['4560','4561','4562','5722','5723','5724','5728','K704','K711','K721','K729',
                                                               'K765','K766',
                                                                 'K767','I850','I859','I864','I982'],3],
                        'metastatic cancer' : [['196','197','198','199','C77','C78','C79','C80'],6],
                        'aids' : [['042','043','044','B20','B21','B22','B24'],6]
}



In [19]:
condition_df = pd.DataFrame.from_dict(condition_dictionary,orient='index',columns=columns)
condition_df.head()

,icd,charlson87_weight
myocardial infarction,"[410, 412, I21, I22, I252]",1
congestive heart failure,"[39891, 40201, 40211, 40291, 40401, 40403, 404...",1
peripheral vascular disease,"[0930, 4373, 4431, 4432, 4433, 4434, 4435, 443...",1
cerebrovascular disease,"[430, 431, 432, 433, 434, 435, 436, 437, 438, ...",1
dementia,"[29410, 29411, 3312, 290, F051, G311, F00, F01...",1


In [20]:
# #xtract ICD codes
condition_list_icd = condition_df['icd'].to_list()

# Flatten list
condition_list_icd = [item for sublist in condition_list_icd for item in sublist]

# Split list into dictionary by length of code
condition_dictionary_icd = split_list_to_dict(condition_list_icd)

### Query Database

In [21]:
condition_query = ("""\
                    WITH condition_start_filter AS (
                        SELECT vco.*
                        FROM omop.v_condition_occurrence AS vco
                        LEFT join omop.person AS p
                            ON vco.person_id = p.person_id
                        WHERE condition_type_concept_id = 32840
                            AND (vco.condition_start_date - p.birth_datetime::date) >= 0
                    )    
                    SELECT DISTINCT person_id, 
                    CASE WHEN substring(translate(condition_source_value,'.',''),1,4) IN ('4373') THEN '4373' 
                        WHEN substring(condition_source_value,1,3) IN {0} THEN substring(condition_source_value,1,3)
                        WHEN substring(translate(condition_source_value,'.',''),1,4) IN {1} THEN substring(translate(condition_source_value,'.',''),1,4)
                        WHEN substring(translate(condition_source_value,'.',''),1,5) IN {2} THEN substring(translate(condition_source_value,'.',''),1,5)
                        ELSE NULL END AS condition_source_value,
                    condition_source_concept_vocabulary_id,
                    condition_start_date
                    FROM condition_start_filter
                    WHERE CASE
                        WHEN substring(translate(condition_source_value,'.',''),1,4) IN ('4373') THEN  1
                        WHEN substring(condition_source_value,1,3) IN {0} THEN 1
                        WHEN substring(translate(condition_source_value,'.',''),1,4) IN {1} THEN 1 
                        WHEN substring(translate(condition_source_value,'.',''),1,5) IN {2} THEN 1 
                        ELSE 0
                        END = 1;
                    """).format(tuple(condition_dictionary_icd[3]),tuple(condition_dictionary_icd[4]),tuple(condition_dictionary_icd[5]))


In [22]:
column_names = ['person_id','condition_source_value','condition_source_concept_vocabulary_id', 'condition_start_date']

d_types = {
    'person_id': 'object',
    'condition_source_value': 'object',
    'condition_source_concept_vocabulary_id': 'object',
    'condition_start_date': 'datetime64[ns]'
}

person_condition_df = query_to_dataframe(condition_query,column_names,d_types)

### Where condition 437.3 is present, we need to add condition 437

This condition needs to be listed twice in the DataFrame. We don't need to do this for the other duplicated ICD-9 codes because the ```calculate_score``` function deals with those correctly because it is the same code listed twice, not slightly differently. 

In [24]:
person_condition_df = add_new_row(person_condition_df)

### Check that subset of conditions correspond to ICD9, not ICD10

Check for the following conditions: V43.4, V42.7, V42.0, V45.1, V56.x

These can correspond to codes in both ICD9 and ICD10, but for Charlson should be ICD9 codes (per Quan 2005). Therefore, remove rows where the condition is one of those and the source vocabulary is ICD10CM. 

In [25]:
# Filter out rows where condition_source is V42 or V44 and vocabulary_id is ICD10CM
person_condition_df = person_condition_df[~((person_condition_df['condition_source_value'].isin(['V434', 'V427', 'V420','V451','V56'])) & 
                   (person_condition_df['condition_source_concept_vocabulary_id'] == 'ICD10CM'))]

### Link Charlson Conditions to Visits
Connect all inpatient visits to Charlson conditions on the basis of *person_id*, then filter to retain only conditions that started on or before the last day of a visit. 

In [26]:
# Merge the inpatient visits DataFrame with the conditions DataFrame
visit_condition_df = inpatient_visits.merge(person_condition_df, how='left', on='person_id')

# Apply filter - the condition start date must be no later than the last day of the inpatient visit
visit_condition_df_filter = np.where(visit_condition_df['visit_end_date'] >= visit_condition_df['condition_start_date'])
visit_condition_df = visit_condition_df.loc[visit_condition_df_filter]

### Group conditions by visit_occurrence_id

In [27]:
# Use groupby to get a list of all conditions for each inpatient visit
visit_condition_grouped_df = visit_condition_df.groupby('visit_occurrence_id')['condition_source_value'].apply(list).reset_index()

### Calculate CCI based on conditions

In [28]:
# Specify column name for condition weights in the condition_df
weight_column = 'charlson87_weight'

visit_condition_grouped_df['comorbidity_score'] = visit_condition_grouped_df.apply(
    lambda row: calculate_score(row['condition_source_value'],weight_column),axis=1)

### Calculate C-score based on CCI

In [29]:
def c_score_mapping(x):
    '''
    Convert from a CCI score to the C score for LACE. The C score matches the 
    Charlson score up to 3, but any Charlson score above 3 is recorded as a c-score of 5
    
    PARAMETER
    x : int
        CCI score for corresponding visit
        
    RETURNS
    cci_score : int
        score for the C component of LACE
    '''
    if x == 0:
        cci_score = 0
    elif x == 1:
        cci_score = 1
    elif x == 2:
        cci_score = 2
    elif x == 3:
        cci_score = 3
    elif x >= 4:
        cci_score = 5
    else:
        cci_score = 0
    return cci_score

In [30]:
visit_condition_grouped_df['c_score'] = visit_condition_grouped_df.apply(
    lambda row: c_score_mapping(row['comorbidity_score']),axis=1)

# Select only the visit id and the c-score, for joining back to the final DataFrame
charlson_comorbidity_index = visit_condition_grouped_df[["visit_occurrence_id", "c_score"]]

# Calculate E: Emergency Visits in Last 180 Days

While van Walraven et al. (2010), uses ER visits in the past 6 months, 180 days is a more consistent unit to use. ER visits which took place on the same day as the inpatient admission, but did not directly lead to that admission, are counted towards the total. 

In [31]:
def e_score_mapping(x):
    '''
    Calculates the E score based on the number of ER visits in the previous 180 days.
      
    PARAMETERS
    x : int
        number of ER visits in the previous 180 days
    
    RETURNS
    e_score : int
        score for the E component of LACE    
    '''
    if x == 0:
        e_score = 0
    elif x == 1:
        e_score = 1
    elif x == 2:
        e_score = 2
    elif x == 3:
        e_score = 3
    else:
        e_score = 4
    return e_score 

In [32]:
def get_e_score(inpatient_visits, er_visits):
    '''
    Calculates a score based on the number of patient ER visits in the previous 180 days. 
       
    PARAMETERS
    inpatient_visits : DataFrame
        dataframe with only inpatient visits
        
    er_visits : DataFrame
        dataframe with only ER visits
           
    RETURNS
    DataFrame
        dataframe with visit_occurrence_id and the score for the 
        E component of LACE
    '''
    # Rename and select relevant columns from the er_visits DataFrame
    er_visits.loc[:,'er_visit_date'] = er_visits['visit_start_date']
    er_visits = er_visits[["person_id","er_visit_date"]]
    
    # Merge the DataFrames and calculate the difference in days
    # then keep only the visits where the difference is 0 to 180 days
    visits_prior_180_days = inpatient_visits.merge(er_visits, how='left', on='person_id')
    visits_prior_180_days['date_diff'] = visits_prior_180_days.visit_start_date - visits_prior_180_days.er_visit_date
    visits_filtered = np.where((visits_prior_180_days['date_diff'] >= np.timedelta64(0, 'D'))
                              & (visits_prior_180_days['date_diff'] <= np.timedelta64(180, 'D')))
    visits_prior_180_days = visits_prior_180_days.loc[visits_filtered]
    
    # Calculate the visit counts
    visit_counts = visits_prior_180_days.groupby(['visit_occurrence_id','visit_concept_id']).size().reset_index(name='prior_er_visits')
    visit_counts['adjustment'] = visit_counts['visit_concept_id'].apply(lambda x: 1 if x == 262 else 0)
    visit_counts['prior_er_visits'] = visit_counts['prior_er_visits'] - visit_counts ['adjustment']
    
    # Calculate ER visits in the previous 180 days
    # for each inpatient visit
    er_visits_past_180_days = inpatient_visits.merge(visit_counts, how='left', on='visit_occurrence_id')
    er_visits_past_180_days = er_visits_past_180_days.replace(np.NaN, 0)
    er_visits_past_180_days['e_score'] = er_visits_past_180_days['prior_er_visits'].apply(e_score_mapping)
    er_visits_past_180_days = er_visits_past_180_days[["visit_occurrence_id","e_score"]]
    
    return er_visits_past_180_days

In [33]:
er_visits_past_180_days = get_e_score(inpatient_visits, er_visits)

/tmp/ipykernel_146/654260867.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  er_visits.loc[:,'er_visit_date'] = er_visits['visit_start_date']


## Calculate LACE

Combine all scores into a single DataFrame, sum together to get LACE score

In [34]:
# Select columns in inpatient_visits
lace = inpatient_visits[["visit_occurrence_id", "person_id", "visit_concept_id", "visit_start_date"]]

# Merge the four subscores into the DataFrame
lace = lace.merge(length_of_stay, how='left', on='visit_occurrence_id')
lace = lace.merge(acuity_of_admission, how='left', on='visit_occurrence_id')
lace = lace.merge(charlson_comorbidity_index, how='left', on='visit_occurrence_id')
lace = lace.merge(er_visits_past_180_days, how='left', on='visit_occurrence_id')

# For c_score, replace NaN with 0
lace = lace.replace(np.NaN, 0)

# Calculate final LACE score
lace["lace_score"] = lace["l_score"] + lace["a_score"] + lace["c_score"] + lace["e_score"]